In [1]:
import pandas as pd
import numpy as np
import os
import requests
from TheMovieDB_key import KEY
from urllib.parse import quote # this is used to fix illegal characters in the titlenames.
from sqlalchemy import create_engine

In [2]:
## Resaved the metacritic_movies.csv to get rid of commas in the numbers with Excel.

# Created schema for movies_t. Imported data into table just to make sure it worked, then deleted the table, and recreated
# to be an empty table.

In [3]:
# load the movies data.
csv_file = "Metacritic Reviews/metacritic_movies.csv"
raw_movies_df = pd.read_csv(csv_file, encoding="UTF-8")
#raw_movies_df = pd.read_csv(csv_file, encoding="latin_1")
raw_movies_df.head()

,movie_title,release_date,genre,meta_mixed,meta_negative,meta_positive,metascore,user_mixed,user_negative,user_positive,userscore
0,Anatomy of a Murder,1-Jul-59,"Drama,Mystery,Thriller,Crime",0,0,15,95,0,0,3,tbd
1,Bringing Up Baby,18-Feb-38,"Comedy,Romance,Family",0,1,16,91,1,0,2,tbd
2,After Life,12-May-99,"Drama,Fantasy",0,0,19,91,0,2,1,tbd
3,Gavagai,3-Aug-18,Drama,1,0,6,91,0,1,2,tbd
4,The Hustler,25-Sep-61,"Drama,Sport",1,0,17,90,0,0,3,tbd


In [4]:
## metacritic_reviews.csv data cleaning

# Previewed in Excel.
# Removed lines of invalid data that appears to have wrapped incorrectly.

# Needed to fix text for the apostrophes that have been convered to â€™
# Did Find/Replace using text editor.
 
# created file Cleaned_metacritic_reviews.csv

In [5]:
# Load the reviews data.
csv_file = "Metacritic Reviews/Cleaned_metacritic_reviews.csv"
raw_reviews_df = pd.read_csv(csv_file, dtype={3:'str',4:'str'}, encoding="latin_1") # latin-1 because UTF-8 had issues in later steps.
raw_reviews_df.head()

,critic_name,media,movie_title,review_date,individual_meta_score,text
0,Robbie Collin,The Telegraph,Jojo Rabbit,7-Oct-19,20,As satire it's a dismal dereliction of duty; a...
1,Roger Moore,NaN,Carrie Pilby,7-Oct-19,38,Carrie Pilby never rises to the level of hatef...
2,NaN,Empire,Gemini Man,7-Oct-19,40,It gives you two Will Smiths for the price of ...
3,Noel Murray,Los Angeles Times,In the Tall Grass,7-Oct-19,50,Despite the handsome Craig Wrobleski cinematog...
4,Jesse Hassenger,The A.V. Club,The King,7-Oct-19,50,It's hard to feel energized by a historical ep...


In [6]:
## Data cleanup complete

In [7]:
## Addy's section

# Need to create table of movie posters urls from TheMovieDB.org

In [8]:
## Sample
# url = 'https://api.themoviedb.org/3/movie/550?api_key=3d48b9204ca053957eb4375bb97a83c8'

In [9]:
## Sample 
# config_path = 'http://api.themoviedb.org/3/configuration?api_key={key}'
# url = config_path.format(key=KEY)
# print(url)
# r = requests.get(url)
# config = r.json()
# config

In [10]:
## Sample
#base_url = config['images']['base_url']
#base_url

In [11]:
# This is what happens if there's no poster.

temp = 'http://api.themoviedb.org/3/search/movie?query="Besotted"&api_key=' + KEY
r = requests.get(temp)
response = r.json()
if response['results'][0]['poster_path'] is None:
    print('no poster')

no poster


In [12]:
# This is what happens if the movie title is not found.

temp = 'http://api.themoviedb.org/3/search/movie?query="dasgdsf"&api_key=' + KEY
r = requests.get(temp)
response = r.json()
response['total_results']

0

In [13]:
# This is what happens if the movie title has # in it.

temp = 'http://api.themoviedb.org/3/search/movie?query="#Horror"&api_key=' + KEY
r = requests.get(temp)
#print(temp)
response = r.json()
response

{'status_code': 7,
 'status_message': 'Invalid API key: You must be granted a valid key.',
 'success': False}

In [14]:
# This is what happens if the movie title has # in it, and is fixed via urllib.parse's quote function.

movieTitle=quote('#Horror')
temp = 'http://api.themoviedb.org/3/search/movie?query=' + movieTitle + '&api_key=' + KEY
#print(temp)
r = requests.get(temp)
response = r.json()
response['results'][0]

{'popularity': 5.063,
 'vote_count': 123,
 'video': False,
 'poster_path': '/xOIDsTAg8KzjOeUYISDDW533y0y.jpg',
 'id': 301325,
 'adult': False,
 'backdrop_path': None,
 'original_language': 'de',
 'original_title': '#Horror',
 'genre_ids': [18, 27, 9648, 53],
 'title': '#Horror',
 'vote_average': 3.4,
 'overview': 'Inspired by actual events, a group of 12 year old girls face a night of horror when the compulsive addiction of an online social media game turns a moment of cyber bullying into a night of insanity.',
 'release_date': '2015-11-20'}

In [15]:
# not sure why, but this movie is one that has the values in the tuple swapped when .

movieTitle=quote('No Home Movie')
temp = 'http://api.themoviedb.org/3/search/movie?query=' + movieTitle + '&api_key=' + KEY
#print(temp)
r = requests.get(temp)
response = r.json()
response['results'][0]['poster_path']

'/eUL2USo8weZHhNMJuWwKtegJzhL.jpg'

In [16]:
# Reused
# base_url = 'http://image.tmdb.org/t/p/'
# size = 'w500'
# rel_path = '/mgwZvl0DwS1GIborSGlpTGomrJD.jpg'

In [17]:
base_url = 'http://image.tmdb.org/t/p/'
size = 'w500' # We're looking for 500px sized posters

In [18]:
url_list = [] # initialized the list where we're storing tuples of (movie_title, poster_url)

# the following 2 lists aren't needed since we're just writing them to a file.
# movie_not_found_list = []
# poster_not_found_list = []

# Also removing references to writing to the file, url_list.txt

processed_count = len(raw_movies_df)
movies_not_found_count = 0
posters_not_found_count = 0
keyerrors_count = 0

keyerrors_f = open("keyerrors.txt", "w")
movies_not_found_f = open("movies_not_found.txt", "w")
posters_not_found_f = open("posters_not_found.txt", "w")

# url_list_f = open("url_list.txt", "w") 
# url_list_f.write("movie_titie" + "/t" + "poster_url") # this is the header to tab delimited file, url_list.txt

print('-----------------------------')
print('Begin Data Retreival. Rows left.')
print('-----------------------------\n')
for title in raw_movies_df.movie_title:
    #print(title)
    temp = 'http://api.themoviedb.org/3/search/movie?query=' + quote(title) + '&api_key=' + KEY
    #print(temp)
    try:
        r = requests.get(temp)
        response = r.json()
        
        if (response['total_results'] != 0) and (response['results'][0]['poster_path'] is not None):
            url = base_url + size + response['results'][0]['poster_path']
            url_list.append([title, url])
#            url_list_f.write(title + "\t" + url + "\n") # make it Tab delimited instead.
        elif response['total_results'] == 0:
            movies_not_found_f.write(title + '\n')
            movies_not_found_count += 1
        elif response['results'][0]['poster_path'] is None:
            posters_not_found_f.write(title + '\n')
            posters_not_found_count += 1
     
        #Print a period every iteration so that we know this is still running
        processed_count -= 1
        if (processed_count % 25 == 0):
            print(processed_count, end = '') # print the number left on every count divisable by 25.
        print('.', end = '')
        
    except KeyError:
        print(f'\n ** keyerror with {title} ** \n')
        keyerrors_count += 1
        keyerrors_f.write(title + '\n')
    
# close the files.
#url_list_f.close()
keyerrors_f.close()
movies_not_found_f.close()
posters_not_found_f.close()

print('\n\n-----------------------------')
print('Data Retrieval Complete')
print('-----------------------------')
print(f'Movies not found: {movies_not_found_count}')
print(f'Movies with no posters: {posters_not_found_count}')
print(f'Other errors: {keyerrors_count}\n')
print('See the files movies_not_found.txt, posters_not_found.txt, and keyerrors.txt for details.')      
    

-----------------------------
Begin Data Retreival. Rows left.
-----------------------------



....................9100.........................9075.........................9050.........................9025.........................9000.........................8975.........................8950.........................8925.........................8900.........................8875.........................8850.........................8825.........................8800.........................8775.........................8750.........................8725.........................8700.........................8675.........................8650.........................8625.........................8600.........................8575.........................8550.........................8525.........................8500.........................8475.........................8450.........................8425.........................8400.........................8375.........................8350.........................8325.........................8300.........................8275...................

.....2050.........................2025.........................2000.........................1975.........................1950.........................1925.........................1900.........................1875.........................1850.........................1825.........................1800.........................1775.........................1750.........................1725.........................1700.........................1675.........................1650.........................1625.........................1600.........................1575.........................1550.........................1525.........................1500.........................1475.........................1450.........................1425.........................1400.........................1375.........................1350.........................1325.........................1300.........................1275.........................1250.........................1225.........................1200.....

In [19]:
# url_list

In [20]:
## We won't use this method because of the issues with data between swapped between fields.

# Update:  So in the previous for-loop where the posters were looked up, the line for
#   url_list.append([title, url])     was initially url_list.append({title, url}), 
#   using curly braces instead square by accident.

# It didn't throw any errors, but apprently that was the case because it works now.

url_df = pd.DataFrame(url_list, columns = ['movie_title','poster_url'])
url_df.tail(25)

,movie_title,poster_url
8718,Lourdes,http://image.tmdb.org/t/p/w500/8ClmEJGwp9ybyOQ...
8719,A Murder in the Park,http://image.tmdb.org/t/p/w500/g0wME5TyTTdvUXt...
8720,Melancholian 3 huonetta,http://image.tmdb.org/t/p/w500/hq3QgZBHCvuSh5h...
8721,The Adventures of Elmo in Grouchland,http://image.tmdb.org/t/p/w500/qSEPC9sP0vlyPbg...
8722,The Haunting,http://image.tmdb.org/t/p/w500/dU07NZikdFAA5cK...
8723,"Her Majesty, Mrs. Brown",http://image.tmdb.org/t/p/w500/evlNi0DTvG1oR40...
8724,Bird People,http://image.tmdb.org/t/p/w500/moJ3ozCPhj7pSiC...
8725,He Named Me Malala,http://image.tmdb.org/t/p/w500/39c68qAkgIIOYDp...
8726,The World,http://image.tmdb.org/t/p/w500/g5IoYeudx9XBEfw...
8727,The Strange Little Cat,http://image.tmdb.org/t/p/w500/nXsvN2QrwEJXJBN...


In [21]:
## Converting to a dataframe didn't work. So I'm going to read in the text file that was created.

# Narrator: Converting to a dataframe works if you use the right syntax... 
# Me: Thanks, I guess.

# No longer needed.

#url_file = "url_list.txt"
#poster_url_df = pd.read_csv(url_file, delimiter= "\t", header=0, encoding="latin_1")
#poster_url_df.tail(25)        

In [22]:
rds_connection_string = "postgres:postgres@localhost:5432/Metacritic_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [23]:
engine.table_names()

['reviews_t', 'movies_t', 'posters_t']

In [24]:
# Push all dataframes to tables in Metacritic_db

In [25]:
url_df.to_sql(name='posters_t', con=engine, if_exists='append', index=False)

In [26]:
raw_reviews_df.to_sql(name='reviews_t', con=engine, if_exists='append', index=False)

In [27]:
raw_movies_df.to_sql(name='movies_t', con=engine, if_exists='append', index=False)

In [28]:
pd.read_sql_query('select * from posters_t', con=engine).head()

,movie_title,poster_url
0,Anatomy of a Murder,http://image.tmdb.org/t/p/w500/lMWWs7IskGldLfw...
1,Bringing Up Baby,http://image.tmdb.org/t/p/w500/pxgUiiP29ysQ4cu...
2,After Life,http://image.tmdb.org/t/p/w500/iTJUqvnF2Q9BRt8...
3,Gavagai,http://image.tmdb.org/t/p/w500/mgwZvl0DwS1GIbo...
4,The Hustler,http://image.tmdb.org/t/p/w500/n2zbi9ePKNkCggu...


In [29]:
pd.read_sql_query('select * from movies_t', con=engine).head()

,movie_title,release_date,genre,meta_mixed,meta_negative,meta_positive,metascore,user_mixed,user_negative,user_positive,userscore
0,Anatomy of a Murder,2059-07-01,"Drama,Mystery,Thriller,Crime",0,0,15,95,0,0,3,tbd
1,Bringing Up Baby,2038-02-18,"Comedy,Romance,Family",0,1,16,91,1,0,2,tbd
2,After Life,1999-05-12,"Drama,Fantasy",0,0,19,91,0,2,1,tbd
3,Gavagai,2018-08-03,Drama,1,0,6,91,0,1,2,tbd
4,The Hustler,2061-09-25,"Drama,Sport",1,0,17,90,0,0,3,tbd


In [30]:
pd.read_sql_query('select * from reviews_t', con=engine).head()

,critic_name,media,movie_title,review_date,individual_meta_score,text
0,Robbie Collin,The Telegraph,Jojo Rabbit,2019-10-07,20.0,As satire it's a dismal dereliction of duty; a...
1,Roger Moore,None,Carrie Pilby,2019-10-07,38.0,Carrie Pilby never rises to the level of hatef...
2,None,Empire,Gemini Man,2019-10-07,40.0,It gives you two Will Smiths for the price of ...
3,Noel Murray,Los Angeles Times,In the Tall Grass,2019-10-07,50.0,Despite the handsome Craig Wrobleski cinematog...
4,Jesse Hassenger,The A.V. Club,The King,2019-10-07,50.0,It's hard to feel energized by a historical ep...


In [31]:
## Sample query, pulling from all 3 tables.

pd.read_sql_query('select r.movie_title, r.critic_name, r.media, r.text, m.metascore, p.poster_url\
    from reviews_t as r\
    join movies_t as m on m.movie_title = r.movie_title\
    left join posters_t as p on p.movie_title = r.movie_title\
    group by r.movie_title, r.critic_name, r.media, r.text, m.metascore, p.poster_url\
    order by r.movie_title;', con=engine).head()

,movie_title,critic_name,media,text,metascore,poster_url
0,'71,Amy Nicholson,Village Voice,"[An] excellent, tensely controlled thriller.",83,http://image.tmdb.org/t/p/w500/2ZfUpDXEFQxEyXf...
1,'71,Andrew O'Hehir,Salon,"It's a riveting, man-on-the-run genre movie, a...",83,http://image.tmdb.org/t/p/w500/2ZfUpDXEFQxEyXf...
2,'71,Andrew Pulver,The Guardian,It's a film that holds you in a vice-like grip...,83,http://image.tmdb.org/t/p/w500/2ZfUpDXEFQxEyXf...
3,'71,Anthony Lane,The New Yorker,"As the camera darts down alleyways, or prowls ...",83,http://image.tmdb.org/t/p/w500/2ZfUpDXEFQxEyXf...
4,'71,Ben Nicholson,CineVue,'71 is a pulse-raising actioner that stumbles ...,83,http://image.tmdb.org/t/p/w500/2ZfUpDXEFQxEyXf...


In [32]:
# Done